<a href="https://colab.research.google.com/github/sunil-karki/NLP-Question_Classification/blob/master/NLP-QC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## https://www.kaggle.com/c/quora-insincere-questions-classification

* Based on SRK's kernel: https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
* Vladimir Demidov's 2DCNN textClassifier: https://www.kaggle.com/yekenot/2dcnn-textclassifier
* Attention layer from Khoi Ngyuen: https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
* LSTM model from Strideradu: https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
* https://www.kaggle.com/danofer/different-embeddings-with-attention-fork

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c quora-insincere-questions-classification

  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 67.7MB/s]
 82% 13.0M/15.8M [00:00<00:00, 17.0MB/s]
100% 15.8M/15.8M [00:00<00:00, 20.3MB/s]
100% 5.95G/5.96G [06:41<00:00, 20.1MB/s]
100% 5.96G/5.96G [06:42<00:00, 15.9MB/s]
100% 54.9M/54.9M [00:04<00:00, 14.5MB/s]



In [6]:
!ls

embeddings.zip			paragram_300_sl999	   train.csv.zip
glove.840B.300d			sample_data		   wiki-news-300d-1M
GoogleNews-vectors-negative300	sample_submission.csv.zip
kaggle.json			test.csv.zip


In [5]:
!unzip embeddings.zip

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  


In [7]:
!ls

embeddings.zip			paragram_300_sl999	   train.csv.zip
glove.840B.300d			sample_data		   wiki-news-300d-1M
GoogleNews-vectors-negative300	sample_submission.csv.zip
kaggle.json			test.csv.zip


In [7]:
!ls wiki-news-300d-1M

wiki-news-300d-1M.vec


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [0]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 95000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

In [12]:
import time
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)

In [12]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   50G   53G  49% /
tmpfs            64M     0   64M   0% /dev
tmpfs            13G     0   13G   0% /sys/fs/cgroup
tmpfs            13G  8.0K   13G   1% /var/colab
/dev/sda1       114G   51G   64G  44% /etc/hosts
shm              13G  4.0K   13G   1% /dev/shm
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware


In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [0]:
def load_and_prec():
    train_df = pd.read_csv('train.csv.zip', compression='zip') 
    test_df = pd.read_csv('test.csv.zip', compression='zip') 
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    ## split to train and val
    train_df, val_df = train_test_split(train_df, test_size=0.001, random_state=2018)


    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    val_X = val_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    val_y = val_df['target'].values  
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    val_X = val_X[val_idx]
    train_y = train_y[trn_idx]
    val_y = val_y[val_idx]    
    
    return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index

In [15]:
#Just testing
train_df = pd.read_csv('train.csv.zip', compression='zip') 
test_df = pd.read_csv('test.csv.zip', compression='zip') 
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)
train_df[train_df["target"]==1]

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


,qid,question_text,target
22,0000e91571b60c2fb487,Has the United States become the largest dicta...,1
30,00013ceca3f624b09f42,Which babies are more sweeter to their parents...,1
110,0004a7fcb2bf73076489,If blacks support school choice and mandatory ...,1
114,00052793eaa287aff1e1,I am gay boy and I love my cousin (boy). He is...,1
115,000537213b01fd77b58a,Which races have the smallest penis?,1
...,...,...,...
1306093,fffeba722d9b371bd1b9,How is it to have intimate relation with your ...,1
1306094,fffee269360dd0d3947a,Why is it when singers have lyrics about voice...,1
1306099,ffff0e4ea1bb6e16feec,Do pakis smell of curry and shit?,1
1306103,ffff3f0a2449ffe4b9ff,Isn't Trump right after all? Why should the US...,1


In [14]:
#Just testing
# EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
EMBEDDING_FILE = 'glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
#embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

for o in open(EMBEDDING_FILE):
    #print(o)    #Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223  .....
    print(o.split(" "))
'''
#print(embeddings_index)
with open("glove.840B.300d/glove.840B.300d.txt") as myfile:
    head = [next(myfile) for x in range(10)]
    f = open("demofile3.txt", "w")
    f.write(str(head))
    f.close()
print(head)
'''

#embeddings_index = dict(get_coefs(*o.split(" ")) for o in open('demofile3.txt'))
#print(embeddings_index)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['field-group', '0.32129', '-1.6378', '0.75596', '0.26813', '0.47293', '0.26158', '0.85073', '0.29399', '-0.26904', '-2.2186', '0.31221', '-0.50201', '0.11557', '0.24599', '1.3028', '0.36848', '0.84239', '-1.527', '0.37717', '-0.40599', '0.012609', '0.46039', '0.12695', '0.002067', '0.0069745', '-1.0914', '0.29317', '-0.30063', '0.0089122', '-0.11496', '0.40786', '-0.62797', '0.169', '-0.25301', '-0.51534', '0.65677', '-0.99634', '-0.34599', '0.017932', '0.58895', '0.29872', '0.011406', '-0.46086', '-0.53757', '0.2863', '-0.62271', '0.1287', '-1.1615', '-0.6878', '-0.38682', '-0.54675', '0.597', '-0.55561', '0.25024', '1.0137', '1.4105', '-0.25873', '-0.42906', '-0.30526', '-0.22734', '-0.2962', '0.0037484', '0.56168', '-1.1897', '-0.76716', '0.059877', '0.3831', '0.59069', '0.11401', '0.41028', '-0.20433', '-0.24174', '0.28124', '0.86143', '-0.57543', '-1.369', '-1.5294', '0.95703', '1.0324', '-0.44549', '0.3984', '0.092365', '-1.173', '-0.032192', '-0.68014', '0.60831', '0.27091', '0

KeyboardInterrupt: ignored

In [25]:
txt = "Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223"

def get_coefs(word,*arr):
  print(word)
  print(np.asarray(arr, dtype='float32'))
  return word,np.asarray(arr, dtype='float32')


#print(word)
#print(np.asarray(arr, dtype='float32'))
#return word, np.asarray(arr, dtype='float32')

x = txt.split()

print(x)

get_coefs(*txt.split(" "))
print()
#embeddings_index = dict(get_coefs(*txt.split(" ")))

def get_coefs_1(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs_1(*o.split(" ")) for o in open(EMBEDDING_FILE))

print(type(embeddings_index))
print(embeddings_index)

['Glass', '-0.41236', '0.35532', '-0.076236', '0.43268', '0.14769', '-1.1381', '-0.30427', '0.37039', '-0.3804', '0.029443', '-0.081454', '0.15791', '-0.32558', '-0.35445', '0.11918', '-0.13342', '-0.087387', '1.2125', '-0.048223']
Glass
[-0.41236   0.35532  -0.076236  0.43268   0.14769  -1.1381   -0.30427
  0.37039  -0.3804    0.029443 -0.081454  0.15791  -0.32558  -0.35445
  0.11918  -0.13342  -0.087387  1.2125   -0.048223]

Glass
[-0.41236   0.35532  -0.076236  0.43268   0.14769  -1.1381   -0.30427
  0.37039  -0.3804    0.029443 -0.081454  0.15791  -0.32558  -0.35445
  0.11918  -0.13342  -0.087387  1.2125   -0.048223]


ValueError: ignored

In [0]:
txt = "Water -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223\n"

file1 = open("myfile1.txt","a")
file1.write(txt) 
file1.close() 

In [43]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open("myfile1.txt"))
print(type(embeddings_index))
print(embeddings_index)

<class 'dict'>
{'Glass': array([-0.41236 ,  0.35532 , -0.076236,  0.43268 ,  0.14769 , -1.1381  ,
       -0.30427 ,  0.37039 , -0.3804  ,  0.029443, -0.081454,  0.15791 ,
       -0.32558 , -0.35445 ,  0.11918 , -0.13342 , -0.087387,  1.2125  ,
       -0.048223], dtype=float32), 'Water': array([-0.41236 ,  0.35532 , -0.076236,  0.43268 ,  0.14769 , -1.1381  ,
       -0.30427 ,  0.37039 , -0.3804  ,  0.029443, -0.081454,  0.15791 ,
       -0.32558 , -0.35445 ,  0.11918 , -0.13342 , -0.087387,  1.2125  ,
       -0.048223], dtype=float32)}


In [35]:
txt = "Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223"
embeddings_index = dict(get_coefs(*txt.split(" "))
#print(type(embeddings_index))
print(embeddings_index)

SyntaxError: ignored

In [44]:
def get_coefs(word,*arr):
  print(word)
  print(np.asarray(arr, dtype='float32'))
  return word, np.asarray(arr, dtype='float32')


embeddings_index = dict(get_coefs(*o.split(" ")) for o in open("myfile1.txt"))
print(type(embeddings_index))
print(embeddings_index)

Glass
[-0.41236   0.35532  -0.076236  0.43268   0.14769  -1.1381   -0.30427
  0.37039  -0.3804    0.029443 -0.081454  0.15791  -0.32558  -0.35445
  0.11918  -0.13342  -0.087387  1.2125   -0.048223]
Water
[-0.41236   0.35532  -0.076236  0.43268   0.14769  -1.1381   -0.30427
  0.37039  -0.3804    0.029443 -0.081454  0.15791  -0.32558  -0.35445
  0.11918  -0.13342  -0.087387  1.2125   -0.048223]
<class 'dict'>
{'Glass': array([-0.41236 ,  0.35532 , -0.076236,  0.43268 ,  0.14769 , -1.1381  ,
       -0.30427 ,  0.37039 , -0.3804  ,  0.029443, -0.081454,  0.15791 ,
       -0.32558 , -0.35445 ,  0.11918 , -0.13342 , -0.087387,  1.2125  ,
       -0.048223], dtype=float32), 'Water': array([-0.41236 ,  0.35532 , -0.076236,  0.43268 ,  0.14769 , -1.1381  ,
       -0.30427 ,  0.37039 , -0.3804  ,  0.029443, -0.081454,  0.15791 ,
       -0.32558 , -0.35445 ,  0.11918 , -0.13342 , -0.087387,  1.2125  ,
       -0.048223], dtype=float32)}


**Load embeddings**

In [0]:
def load_glove(word_index):
    # EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    EMBEDDING_FILE = 'glove.840B.300d/glove.840B.300d.txt'
    ## This file contains content as following
    #Glass -0.41236 0.35532 -0.076236 0.43268 0.14769 -1.1381 -0.30427 0.37039 -0.3804 0.029443 -0.081454 0.15791 -0.32558 -0.35445 0.11918 -0.13342 -0.087387 1.2125 -0.048223  .....
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    # EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    EMBEDDING_FILE = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    # EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    EMBEDDING_FILE = 'paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]
    print(emb_mean,emb_std,"para")

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

**CNN Model**

In [0]:
# https://www.kaggle.com/yekenot/2dcnn-textclassifier
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()
    
    return model

**Attention layer**

In [0]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

**LSTM models**

In [0]:
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
def model_gru_srk_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    x = Attention(maxlen)(x) # New
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model    
    

In [0]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
def model_gru_atten_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

**Train and predict**

In [0]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def train_pred(model, epochs=2):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y))
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

**Main part: load, train, pred and blend**

In [24]:
train_X, val_X, test_X, train_y, val_y, word_index = load_and_prec()
vocab = []
for w,k in word_index.items():
    vocab.append(w)
    if k >= max_features:
        break
embedding_matrix_1 = load_glove(word_index)
# embedding_matrix_2 = load_fasttext(word_index)
embedding_matrix_3 = load_para(word_index)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


-0.0053247833 0.49346462 para


In [25]:
model_cnn(embedding_matrix_1)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 70, 300)      28500000    input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 70, 300, 1)   0           embedding_1[0][0]                
___________________________________________________________________

In [0]:
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
np.shape(embedding_matrix)

(95000, 300)

In [0]:
outputs = []

In [0]:
pred_val_y, pred_test_y = train_pred(model_gru_atten_3(embedding_matrix), epochs = 3)
outputs.append([pred_val_y, pred_test_y, '3 GRU w/ atten'])

InvalidArgumentError: ignored

In [0]:
pred_val_y, pred_test_y = train_pred(model_gru_srk_atten(embedding_matrix), epochs = 2)
outputs.append([pred_val_y, pred_test_y, 'gru atten srk'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_cnn(embedding_matrix), epochs = 2)
outputs.append([pred_val_y, pred_test_y, '2d CNN'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_cnn(embedding_matrix_1), epochs = 2) # GloVe only
outputs.append([pred_val_y, pred_test_y, '2d CNN GloVe'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_lstm_du(embedding_matrix), epochs = 2)
outputs.append([pred_val_y, pred_test_y, 'LSTM DU'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix), epochs = 3)
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix_1), epochs = 3) # Only GloVe
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention GloVe'])

In [0]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix_3), epochs = 3) # Only Para
outputs.append([pred_val_y, pred_test_y, '2 LSTM w/ attention Para'])

In [0]:
# pred_test_y = np.sum([outputs[i][1] * weights[i] for i in range(len(outputs))], axis = 0)
pred_test_y = np.mean([outputs[i][1] for i in range(len(outputs))], axis = 0)

pred_test_y = (pred_test_y > 0.33).astype(int)
test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)

In [0]:
## https://www.kaggle.com/suicaokhoailang/beating-the-baseline-with-one-weird-trick-0-691
## Khoi Nguyen